### Training and updating models
You can usually make the model more accurate by showing it examples from your domain.
You often also want to predict categories specific to your problem, so the model needs to learn about them.
This is essential for text classification, very useful for entity recognition and a little less critical for tagging and parsing.